In [25]:
from keras.models import load_model
from build_train_set import buildTrainSet
from post_transcription import returnPvLabels, removePvSilence
from load_midi_set import importMidiFiles
from subsequence_dtw import subsequenceDTW, meanReciprocalRank
import matplotlib.pyplot as plt
import numpy as np
import os

cur_path = os.path.abspath(os.path.dirname(''))

## Initialize Dataset

In [2]:
x_train, y_train, label_train, x_test, y_test, label_test, x_tune, y_tune, label_tune = buildTrainSet()
print((x_test.shape, y_test.shape), '==>', len(label_test))

((396, 250, 1), (396, 250, 13)) ==> 396


In [14]:
midi = importMidiFiles()
print(len(midi))

48


## 1. Using LSTM

In [3]:
model = load_model('../ext/model_lstm.h5')
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 250, 256)          264192    
                                                                 
 lstm_6 (LSTM)               (None, 250, 256)          525312    
                                                                 
 lstm_7 (LSTM)               (None, 250, 256)          525312    
                                                                 
 lstm_8 (LSTM)               (None, 250, 256)          525312    
                                                                 
 dropout_2 (Dropout)         (None, 250, 256)          0         
                                                                 
 dense_2 (Dense)             (None, 250, 13)           3341      
                                                                 
Total params: 1,843,469
Trainable params: 1,843,469
No

In [8]:
y_pred = model.predict(x_test)
print('shape:', y_pred.shape)

13/13 [==============================] - 13s 993ms/step
shape: (396, 250, 13)


In [15]:
yp_labels = removePvSilence(returnPvLabels(y_pred))
ya_labels = removePvSilence(returnPvLabels(y_test))

print('len:', len(yp_labels))

len: 396


In [27]:
# Similarity Matching
ranks = []

for i in range (len(yp_labels)):
    costs = []
    for j in range(len(midi)):
        print('Hum ' + str(i+1) + '/' + str(len(yp_labels)) + ' - Song ' + str(j+1) + '/' + str(len(midi)))
        costs.append([j, subsequenceDTW(ya_labels[i], midi[j])])

    costs.sort(key=lambda x: x[1])
    ranks.append(np.array(costs).T.tolist()[0])

    np.save((os.path.join(cur_path, '../ext/ranks')), ranks)

Hum 1/396 - Song 1/48
Hum 1/396 - Song 2/48
Hum 1/396 - Song 3/48
Hum 1/396 - Song 4/48
Hum 1/396 - Song 5/48
Hum 1/396 - Song 6/48
Hum 1/396 - Song 7/48
Hum 1/396 - Song 8/48
Hum 1/396 - Song 9/48
Hum 1/396 - Song 10/48
Hum 1/396 - Song 11/48
Hum 1/396 - Song 12/48
Hum 1/396 - Song 13/48
Hum 1/396 - Song 14/48
Hum 1/396 - Song 15/48
Hum 1/396 - Song 16/48
Hum 1/396 - Song 17/48
Hum 1/396 - Song 18/48
Hum 1/396 - Song 19/48
Hum 1/396 - Song 20/48
Hum 1/396 - Song 21/48
Hum 1/396 - Song 22/48
Hum 1/396 - Song 23/48
Hum 1/396 - Song 24/48
Hum 1/396 - Song 25/48
Hum 1/396 - Song 26/48
Hum 1/396 - Song 27/48
Hum 1/396 - Song 28/48
Hum 1/396 - Song 29/48
Hum 1/396 - Song 30/48
Hum 1/396 - Song 31/48
Hum 1/396 - Song 32/48
Hum 1/396 - Song 33/48
Hum 1/396 - Song 34/48
Hum 1/396 - Song 35/48
Hum 1/396 - Song 36/48
Hum 1/396 - Song 37/48
Hum 1/396 - Song 38/48
Hum 1/396 - Song 39/48
Hum 1/396 - Song 40/48
Hum 1/396 - Song 41/48
Hum 1/396 - Song 42/48
Hum 1/396 - Song 43/48
Hum 1/396 - Song 44/

In [28]:
for i in range (len(ranks)):
    print('----- ' + str(i+1) + '/' + str(len(ranks)) + ' -----')
    print('Label        :', label_test[i])
    print('Ranking      :', ranks[i])
    print('Posisi label : '+ str(ranks[i].index(label_test[i]) + 1) + '/' + str(len(ranks[i])))

----- 1/396 -----
Label        : 34
Ranking      : [34.0, 0.0, 32.0, 38.0, 23.0, 12.0, 1.0, 42.0, 5.0, 45.0, 46.0, 24.0, 33.0, 35.0, 47.0, 13.0, 27.0, 16.0, 20.0, 36.0, 8.0, 28.0, 3.0, 22.0, 18.0, 19.0, 21.0, 30.0, 10.0, 37.0, 40.0, 25.0, 29.0, 41.0, 9.0, 17.0, 6.0, 7.0, 15.0, 26.0, 2.0, 11.0, 39.0, 14.0, 31.0, 43.0, 44.0, 4.0]
Posisi label : 1/48
----- 2/396 -----
Label        : 37
Ranking      : [37.0, 28.0, 24.0, 42.0, 9.0, 25.0, 22.0, 4.0, 10.0, 17.0, 19.0, 21.0, 5.0, 8.0, 46.0, 1.0, 30.0, 45.0, 27.0, 43.0, 47.0, 6.0, 34.0, 0.0, 13.0, 44.0, 3.0, 7.0, 11.0, 16.0, 18.0, 33.0, 12.0, 20.0, 41.0, 32.0, 26.0, 23.0, 35.0, 36.0, 38.0, 31.0, 2.0, 40.0, 15.0, 39.0, 14.0, 29.0]
Posisi label : 1/48
----- 3/396 -----
Label        : 40
Ranking      : [6.0, 32.0, 24.0, 18.0, 8.0, 5.0, 27.0, 1.0, 25.0, 47.0, 42.0, 20.0, 10.0, 30.0, 41.0, 3.0, 46.0, 29.0, 40.0, 19.0, 33.0, 2.0, 15.0, 16.0, 31.0, 28.0, 34.0, 45.0, 23.0, 22.0, 38.0, 39.0, 12.0, 7.0, 13.0, 43.0, 26.0, 37.0, 17.0, 21.0, 14.0, 35.0, 0.0

In [29]:
mrr = meanReciprocalRank(ranks, label_test)
print(mrr)

[34.0, 0.0, 32.0, 38.0, 23.0, 12.0, 1.0, 42.0, 5.0, 45.0, 46.0, 24.0, 33.0, 35.0, 47.0, 13.0, 27.0, 16.0, 20.0, 36.0, 8.0, 28.0, 3.0, 22.0, 18.0, 19.0, 21.0, 30.0, 10.0, 37.0, 40.0, 25.0, 29.0, 41.0, 9.0, 17.0, 6.0, 7.0, 15.0, 26.0, 2.0, 11.0, 39.0, 14.0, 31.0, 43.0, 44.0, 4.0]
34
[37.0, 28.0, 24.0, 42.0, 9.0, 25.0, 22.0, 4.0, 10.0, 17.0, 19.0, 21.0, 5.0, 8.0, 46.0, 1.0, 30.0, 45.0, 27.0, 43.0, 47.0, 6.0, 34.0, 0.0, 13.0, 44.0, 3.0, 7.0, 11.0, 16.0, 18.0, 33.0, 12.0, 20.0, 41.0, 32.0, 26.0, 23.0, 35.0, 36.0, 38.0, 31.0, 2.0, 40.0, 15.0, 39.0, 14.0, 29.0]
37
[6.0, 32.0, 24.0, 18.0, 8.0, 5.0, 27.0, 1.0, 25.0, 47.0, 42.0, 20.0, 10.0, 30.0, 41.0, 3.0, 46.0, 29.0, 40.0, 19.0, 33.0, 2.0, 15.0, 16.0, 31.0, 28.0, 34.0, 45.0, 23.0, 22.0, 38.0, 39.0, 12.0, 7.0, 13.0, 43.0, 26.0, 37.0, 17.0, 21.0, 14.0, 35.0, 0.0, 36.0, 4.0, 44.0, 11.0, 9.0]
40
[9.0, 22.0, 1.0, 34.0, 37.0, 25.0, 45.0, 20.0, 21.0, 7.0, 44.0, 16.0, 0.0, 33.0, 43.0, 11.0, 18.0, 42.0, 47.0, 26.0, 10.0, 38.0, 24.0, 30.0, 17.0, 19.0, 2